In [ ]:
import cv2 
import torch
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import numpy as np
import pandas as pd

from torchvision.models.detection.faster_rcnn  import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
device='cuda' if torch.cuda.is_available() else 'cpu'
base_dir='../input/tensorflow-great-barrier-reef/train_images'

In [ ]:
df_train=pd.read_csv('../input/objdet/apples.csv')

In [ ]:
class ReefDataset:
    def __init__(self,df,transforms=None):
        self.df=df
        self.image_ids=df['video_frame'].unique()
        self.transforms=transforms
        self.unique_image=self.df.video_frame
        self.base_dir='../input/tensorflow-great-barrier-reef/train_images'
        
    def __getitem__(self,ix):
        
        image_id=self.image_ids[ix]
       
        records=self.df[self.df['video_frame']==image_id]
        
        z=self.df[self.df['video_frame']==image_id].video_id.unique().astype(str)
        image=cv2.imread(f'{self.base_dir}/video_{z[0]}/{image_id}.jpg',1)
        
        
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        height=self.df.height[ix]
        boxes=records[['x','y','width','height']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        
        
        # there is only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target['masks'] = None
        target['image_id'] = torch.tensor([ix])
        target['area'] = area
        target['iscrowd'] = iscrowd
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]
        
        
        
      
    

In [ ]:
# Albumentations
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


In [ ]:

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import torchvision

In [ ]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = ReefDataset(df_train, get_train_transform())
#valid_dataset = WheatDataset(valid_df, DIR_TRAIN, get_valid_transform())
 

# split the dataset in train and test set
#indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)
'''
valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)'''

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = None

num_epochs = 2

In [ ]:
loss_hist = Averager()
itr = 1
model=model.float()
for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loader:
        
        images = list(image.float().to(device) for image in images)
        targets = [{k: v.to(torch.float32).to(device) if "box" in k else v.to(device) for k, v in t.items()} for t in targets]
        
        
        loss_dict=model(images,targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 1 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")   

In [ ]:

def store_model(save_path, model, optimizer):
    '''
    Store The weights of model
    
    @Param:
    - save_path(str/path): Path to save your model weights
    - model(nn.Module): Neural Network model to save
    - optimizer(nn.optim): Optimizer Information
    '''
    torch.save({'model_state_dict': model.state_dict(), 
                'optimizer': optimizer.state_dict()}, save_path)

In [ ]:
save_name = '/kaggle/working/save.pth'

In [ ]:
 store_model(save_name, model, optimizer)
